# Clustering Analysis
## Productivity Zone Identification

This notebook performs clustering analysis to identify productivity zones for agricultural states.

**Objective:** Group states into productivity zones based on yield, production, and cost patterns.


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Clustering libraries
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA

# Add src to path
sys.path.append('../src')
from utils.data_loader import load_data, preprocess_data
from clustering.kmeans_clustering import ProductivityZoneClusterer

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")


## 1. Load and Prepare Data


In [ ]:
# Load and preprocess data
df = load_data()
df_processed = preprocess_data(df)

# Aggregate data by state for clustering
if 'State' in df_processed.columns:
    state_aggregated = df_processed.groupby('State').agg({
        'Quantity': 'mean',  # Average yield
        'Production': 'sum',  # Total production
        'Cost': 'mean'  # Average cost
    }).reset_index()
    
    print(f"States to cluster: {len(state_aggregated)}")
    print(f"\nState Statistics:")
    print(state_aggregated.describe())
    
    # Prepare features for clustering
    features = ['Quantity', 'Production', 'Cost']
    X_cluster = state_aggregated[features].values
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_cluster)
    
    print(f"\nFeatures for clustering: {features}")
    print(f"Scaled data shape: {X_scaled.shape}")
    
    state_aggregated.head()
else:
    print("⚠️ 'State' column not found")


## 2. Determine Optimal Number of Clusters (Elbow Method)


In [ ]:
# Elbow method to find optimal number of clusters
inertias = []
silhouette_scores = []
K_range = range(2, 8)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Visualize elbow method
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(K_range, inertias, marker='o', linewidth=2, markersize=8)
axes[0].set_title('Elbow Method for Optimal k', fontweight='bold')
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(K_range)

axes[1].plot(K_range, silhouette_scores, marker='s', linewidth=2, markersize=8, color='green')
axes[1].set_title('Silhouette Score vs Number of Clusters', fontweight='bold')
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(K_range)

plt.tight_layout()
plt.show()

# Find optimal k (highest silhouette score)
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k} (Silhouette Score: {max(silhouette_scores):.4f})")


## 3. K-Means Clustering


In [ ]:
# Perform K-Means clustering with optimal k
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Add cluster labels to dataframe
state_aggregated['Zone'] = cluster_labels
state_aggregated['Zone'] = state_aggregated['Zone'].astype(str)

print(f"Clustering completed with {optimal_k} zones")
print(f"\nStates per zone:")
print(state_aggregated['Zone'].value_counts().sort_index())

# Calculate cluster metrics
silhouette = silhouette_score(X_scaled, cluster_labels)
davies_bouldin = davies_bouldin_score(X_scaled, cluster_labels)

print(f"\nClustering Metrics:")
print(f"  Silhouette Score: {silhouette:.4f} (higher is better)")
print(f"  Davies-Bouldin Index: {davies_bouldin:.4f} (lower is better)")

# Characterize each zone
print("\nZone Characteristics:")
for zone in sorted(state_aggregated['Zone'].unique()):
    zone_data = state_aggregated[state_aggregated['Zone'] == zone]
    print(f"\nZone {zone}:")
    print(f"  Number of states: {len(zone_data)}")
    print(f"  Average Yield: {zone_data['Quantity'].mean():.2f} Quintals/Hectare")
    print(f"  Total Production: {zone_data['Production'].sum():,.0f} Tons")
    print(f"  Average Cost: {zone_data['Cost'].mean():.2f}")
    print(f"  States: {', '.join(zone_data['State'].tolist())}")


## 4. Visualize Clusters


In [ ]:
# Visualize clusters using PCA for 2D projection
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# PCA visualization
scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, 
                          cmap='viridis', s=100, alpha=0.6, edgecolors='black')
axes[0].scatter(kmeans.cluster_centers_[:, 0] if hasattr(kmeans.cluster_centers_, 'shape') 
                else pca.transform(kmeans.cluster_centers_)[:, 0],
                kmeans.cluster_centers_[:, 1] if hasattr(kmeans.cluster_centers_, 'shape')
                else pca.transform(kmeans.cluster_centers_)[:, 1],
                c='red', marker='x', s=200, linewidths=3, label='Centroids')
axes[0].set_title('Clusters (PCA Projection)', fontweight='bold', fontsize=14)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Zone comparison
zone_stats = state_aggregated.groupby('Zone').agg({
    'Quantity': 'mean',
    'Production': 'sum',
    'Cost': 'mean'
}).reset_index()

x = np.arange(len(zone_stats))
width = 0.25

axes[1].bar(x - width, zone_stats['Quantity'], width, label='Avg Yield', color='skyblue')
axes[1].bar(x, zone_stats['Production'] / zone_stats['Production'].max(), width, 
           label='Total Production (normalized)', color='lightgreen')
axes[1].bar(x + width, zone_stats['Cost'] / zone_stats['Cost'].max(), width,
           label='Avg Cost (normalized)', color='coral')

axes[1].set_title('Zone Comparison', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Zone')
axes[1].set_ylabel('Normalized Values')
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'Zone {z}' for z in zone_stats['Zone']])
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 5. Zone-Based Recommendations


In [ ]:
# Generate crop recommendations for each zone
if 'Crop' in df_processed.columns:
    # Get best performing crops per zone
    zone_recommendations = []
    
    for zone in sorted(state_aggregated['Zone'].unique()):
        zone_states = state_aggregated[state_aggregated['Zone'] == zone]['State'].tolist()
        zone_data = df_processed[df_processed['State'].isin(zone_states)]
        
        # Find top crops by average yield in this zone
        crop_performance = zone_data.groupby('Crop').agg({
            'Quantity': 'mean',
            'Production': 'sum'
        }).sort_values('Quantity', ascending=False)
        
        zone_recommendations.append({
            'Zone': zone,
            'States': ', '.join(zone_states),
            'Top Crops': ', '.join(crop_performance.head(5).index.tolist()),
            'Avg Yield': crop_performance['Quantity'].mean()
        })
    
    recommendations_df = pd.DataFrame(zone_recommendations)
    print("Zone-Based Crop Recommendations:")
    print(recommendations_df.to_string(index=False))
    
    # Visualize recommendations
    fig, ax = plt.subplots(figsize=(12, 6))
    top_crops_per_zone = []
    for zone in sorted(state_aggregated['Zone'].unique()):
        zone_states = state_aggregated[state_aggregated['Zone'] == zone]['State'].tolist()
        zone_data = df_processed[df_processed['State'].isin(zone_states)]
        crop_performance = zone_data.groupby('Crop')['Quantity'].mean().sort_values(ascending=False)
        top_crops_per_zone.append(crop_performance.head(3).index.tolist())
    
    # Create visualization
    y_pos = np.arange(len(state_aggregated['Zone'].unique()))
    colors = plt.cm.viridis(np.linspace(0, 1, len(state_aggregated['Zone'].unique())))
    
    for i, (zone, crops) in enumerate(zip(sorted(state_aggregated['Zone'].unique()), top_crops_per_zone)):
        ax.barh(i, len(crops), color=colors[i], label=f'Zone {zone}')
        ax.text(len(crops)/2, i, ', '.join(crops), ha='center', va='center', fontweight='bold')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f'Zone {z}' for z in sorted(state_aggregated['Zone'].unique())])
    ax.set_xlabel('Number of Recommended Crops')
    ax.set_title('Top Crops by Zone', fontweight='bold', fontsize=14)
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 'Crop' column not found for recommendations")


## 6. Save Clustering Results


In [ ]:
# Save clustering results
models_dir = Path("../models/saved_models")
models_dir.mkdir(parents=True, exist_ok=True)

# Save clusterer
clusterer = ProductivityZoneClusterer(n_clusters=optimal_k)
clusterer.fit(df_processed)
clusterer.save(str(models_dir / "kmeans_clusterer.joblib"))
print("✅ Clustering model saved")

# Save zone assignments
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)
state_aggregated.to_csv(output_dir / "state_zones.csv", index=False)
print("✅ Zone assignments saved to data/processed/state_zones.csv")

print(f"\nClustering results saved successfully!")


## 7. Summary

### Clustering Results:
- **Number of Zones:** [Optimal k]
- **Silhouette Score:** [Value]
- **Davies-Bouldin Index:** [Value]

### Zone Characteristics:
- **Zone 0:** [Description - High/Medium/Low productivity]
- **Zone 1:** [Description]
- **Zone 2:** [Description]
- [Add more zones as needed]

### Key Insights:
- [Add insights about zone patterns]
- [Add observations about state groupings]
- [Add recommendations for zone-based crop selection]

### Next Steps:
- Use zones for crop recommendations in API
- Integrate zone information into dashboard
- Refine clustering based on additional features
